In [2]:
# Keras is a library that acts like a wrapper around TensorFlow, provides simple commands to use TensorFlow

# keras.datasets is a library of pre-packaged sample data
# Import MNIST database, collection of handwritten digits - the "real" data
from keras.datasets import mnist

# A layer is a specific mathematical transformation that data passes through
# Input - a placeholder for the starting data
# Dense - a standard brick; every neuron in this layer connects to every neuron in the next
# Reshape - a morphing brick; changes the data from a flat line into a 2D square
# Dropout - a filter brick; randomly turns off neurons to prevent the model from overfitting
from keras.layers import Input, Dense, Reshape, Flatten

# Standardies the outputs from the previous layer to always have a mean of 0 and a standard deviation of 1
from keras.layers import BatchNormalization

# Standard ReLu turns all negative numbers into zero
# Leaky ReLu allows a small percentage of negative signal to pass through, keeps the GAN "alive"
# Original line - from keras.layers.advanced_activations import LeakyReLU
from keras.layers import LeakyReLU

# Defines the "container" for the layers
# Sequential - A simple, linear stack what data flows through each layer
# Model - More flexible paths
from keras.models import Sequential, Model

# The optimizer is the algorithm that updates the model's weights to make it better; Adam is a popular choice
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Each image in the MNIST dataset is 28x28 pixels.
# Define input image dimensions
img_rows = 28
img_cols = 28

# The images are in grayscale, so each pixel has 1 value describing how bright it is (black to white)
channels = 1

# Creates a tuple defining our image shape
img_shape = (img_rows, img_cols, channels)

In [7]:
# Given an input of noise (latent) vector, the Generator will generate an image
def build_generator():

	### Sequential API - BUILD THE MODEL ###
  
	# Define the noise (seed) to be a vector (1D) of size 100
	noise_shape = (100,)
  
	# Define the generator network
  # Note: Current GAN is simple - only uses Dense layers (easy to understand, fast to train) - but don't see shapes well
	#       There are more powerful versions (e.g., VGG for super-resolution GAN)
  
	# Creates an empty container to add layers into.
  # Data flows through each layer
	model = Sequential()
  
	# Adds a layer into the model - first layer, so it is in the input layer
	# Creates a Dense layer with 256 neurons, and expect an input of 100 numbers (noise_shape)
	model.add(Dense(256, input_shape=noise_shape))

	# Alpha is a hyperparameter that controls how much the function passes through negative network inputs
	# If alpha is too small, the signal is too weak and the neurons can still die
	# If alpha is too big, the model becomes too linear and loses its inability to learn complex shapes
	model.add(LeakyReLU(alpha=0.2))

	# Batch normalization keeps a running average of mean and variance of the data
	# Momentum decides how much weight to give to the past (80%) vs. the current data (20%)
	# If momentum is too small, the normalization would jump around wildly with each new batch of data
	# Acts as stabilizer, which allows a higher learning rate
	model.add(BatchNormalization(momentum=0.8))

	model.add(Dense(512))
	model.add(LeakyReLU(alpha=0.2))
	model.add(BatchNormalization(momentum=0.8))
	model.add(Dense(1024))
	model.add(LeakyReLU(alpha=0.2))
	model.add(BatchNormalization(momentum=0.8))

	# Each Dense layer adds more parameters (256 -> 512 -> 1024), allowing the model to learn more intricate details.

	# np.prod multiples 28x28x1 (from img_shape) = 784
	# The final Dense layer creates 784 outputs - one for every pixel in the MNIST image
	# tanh activation normalizes all output numbers to be between -1 and 1
	model.add(Dense(np.prod(img_shape), activation='tanh'))

	# Before, the data is a long list of 784 numbers
	# Reshape folds it into a 28x28 grid - turns it into an image
	model.add(Reshape(img_shape))

	# Prints a table showing the layers and number of parameters
	model.summary()


	### Functional API - USE THE MODEL ###

	# Creates the Tensorspace to accept a list of 100 numbers, the size of the seed
	noise = Input(shape=noise_shape)

	# Feeds the noise through the finished model - returns the generated image
	img = model(noise)

	# Wrap the input, layers, and output into a single, usable object called a Model
	return Model(noise, img)